# LineLogic Video Analysis on VastAI
Bu notebook VastAI'da RTX 4090 ile video analizi yapmak için hazırlanmıştır.

## 1. Kurulum ve Bağımlılıklar

In [ ]:
# Sistem bilgilerini kontrol et
!nvidia-smi
!python --version
!pwd

In [ ]:
# Gerekli paketleri yükle
!pip install gdown ultralytics supervision opencv-python numpy torch torchvision

In [ ]:
# PyTorch CUDA durumunu kontrol et
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"GPU count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 2. Proje Kurulumu

In [ ]:
# Çalışma dizinini ayarla
import os
os.chdir('/workspace')

# Proje klonla (eğer yoksa)
if not os.path.exists('line48'):
    !git clone https://github.com/your-username/line48.git
    
os.chdir('/workspace/line48')
!pwd
!ls -la

In [ ]:
# Gerekli dizinleri oluştur
!mkdir -p videos outputs logs models
!ls -la

## 3. Google Drive'dan Video İndirme

In [ ]:
# Google Drive dosya ID'sini çıkar
drive_url = "https://drive.google.com/file/d/1OYdAf3OMYIFLnGAi8Gx9an_xv3BulPpN/view?usp=share_link"
file_id = "1OYdAf3OMYIFLnGAi8Gx9an_xv3BulPpN"

print(f"Dosya ID: {file_id}")

In [ ]:
# gdown ile videoyu indir
import gdown

# Video dosyasını videos dizinine indir
output_path = "videos/input_video.mp4"
gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)

# Dosya boyutunu kontrol et
import os
if os.path.exists(output_path):
    file_size = os.path.getsize(output_path) / (1024*1024)  # MB
    print(f"Video indirildi: {output_path}")
    print(f"Dosya boyutu: {file_size:.1f} MB")
else:
    print("Video indirilemedi!")

## 4. Video Bilgilerini Kontrol Et

In [ ]:
import cv2

# Video bilgilerini al
video_path = "videos/input_video.mp4"
cap = cv2.VideoCapture(video_path)

if cap.isOpened():
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duration = frame_count / fps
    
    print(f"Video Bilgileri:")
    print(f"  Çözünürlük: {width}x{height}")
    print(f"  FPS: {fps}")
    print(f"  Frame sayısı: {frame_count}")
    print(f"  Süre: {duration:.1f} saniye")
    
    cap.release()
else:
    print("Video açılamadı!")

## 5. YOLO Modelini Hazırla

In [ ]:
# YOLO modelini indir ve test et
from ultralytics import YOLO

# Model dosyasını indir (ilk seferde otomatik indirilir)
model = YOLO('yolo11n.pt')  # Nano model (hızlı)
# model = YOLO('yolo11s.pt')  # Small model (daha iyi accuracy)
# model = YOLO('yolo11m.pt')  # Medium model (RTX 4090 için uygun)

print(f"Model yüklendi: {model.model}")
print(f"Model device: {model.device}")

## 6. LineLogic Konfigürasyonu

In [ ]:
# src dizinine geç ve modülleri import et
os.chdir('/workspace/line48/src')

# Python path'e ekle
import sys
sys.path.append('/workspace/line48/src')

# Ana modülleri import et
try:
    from run_analysis import run_analysis
    from utils import load_model
    import config
    print("Modüller başarıyla yüklendi")
except ImportError as e:
    print(f"Modül import hatası: {e}")
    print("Manuel kurulum gerekebilir")

## 7. Video Analizi Çalıştır

In [ ]:
# Video analizi parametreleri
video_path = "../videos/input_video.mp4"

# Analiz parametreleri (RTX 4090 için optimize)
analysis_params = {
    'confidence': 0.25,     # YOLO confidence threshold
    'iou': 0.45,           # NMS IoU threshold  
    'imgsz': 1280,         # Input image size (4090 için yüksek)
    'device': 'cuda:0',    # GPU kullan
    'verbose': True
}

print(f"Analiz başlatılıyor...")
print(f"Video: {video_path}")
print(f"Parametreler: {analysis_params}")

In [ ]:
# Ana analizi çalıştır
try:
    # run_analysis fonksiyonunu çağır
    result = run_analysis(
        source_video_path=video_path,
        use_frame_logic=True,  # Frame-based validation kullan
        **analysis_params
    )
    print("Analiz tamamlandı!")
    print(f"Sonuç: {result}")
    
except Exception as e:
    print(f"Analiz hatası: {e}")
    # Alternatif: Manuel command line çalıştır
    print("Manuel command line ile deneniyor...")
    !python run_analysis.py --video "../videos/input_video.mp4" --frame-logic --confidence 0.25 --iou 0.45 --imgsz 1280

## 8. Sonuçları Kontrol Et

In [ ]:
# Output dosyalarını listele
!ls -la ../outputs/
print("\n" + "="*50)
!ls -la ../logs/

In [ ]:
# Log dosyalarını oku ve analiz et
import pandas as pd
import glob

# En son oluşturulan log dosyasını bul
log_files = glob.glob('../logs/*_results_*.csv')
if log_files:
    latest_log = max(log_files, key=os.path.getctime)
    print(f"Son results dosyası: {latest_log}")
    
    # Sonuçları oku
    try:
        df = pd.read_csv(latest_log)
        print(f"\nSonuç özeti:")
        print(df.head())
        
        # Temel istatistikleri göster
        if 'class' in df.columns and 'count' in df.columns:
            print(f"\nSınıf bazında sayımlar:")
            class_counts = df.groupby('class')['count'].sum()
            print(class_counts)
            
    except Exception as e:
        print(f"Log dosyası okuma hatası: {e}")
else:
    print("Log dosyası bulunamadı")

## 9. GPU Performans İzleme

In [ ]:
# GPU kullanımını kontrol et
!nvidia-smi --query-gpu=utilization.gpu,memory.used,memory.total,temperature.gpu --format=csv

In [ ]:
# PyTorch memory usage
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated(0) / 1024**3
    cached = torch.cuda.memory_reserved(0) / 1024**3
    print(f"GPU Memory - Allocated: {allocated:.2f} GB, Cached: {cached:.2f} GB")
    
    # Memory temizle
    torch.cuda.empty_cache()
    print("GPU cache temizlendi")

## 10. Sonuçları İndir

In [ ]:
# Sonuç dosyalarını zip'le
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

!cd /workspace/line48 && zip -r results_{timestamp}.zip outputs/ logs/
print(f"Sonuçlar ziplenди: results_{timestamp}.zip")

# Dosya boyutunu göster
!ls -lh /workspace/line48/results_{timestamp}.zip